#Content Based Recommendation System

##Inisiasi

In [ ]:
pip install html2text

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 215kB 13.3MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import re
import random
import nltk
import sklearn
import html2text
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

##Preprocessing

In [ ]:
datateknologi =  pd.read_csv('/content/drive/MyDrive/Datasets/Sagara/wp_posts.csv')
datateknologi

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,2,1,2017-12-20 10:13:01,2017-12-20 10:13:01,This is an example page. It's different from a...,Sample Page,NaN,publish,closed,open,NaN,sample-page,NaN,NaN,2017-12-20 10:13:01,2017-12-20 10:13:01,NaN,0,http://dev.sagara.asia/teknologiid/?page_id=2,0,page,NaN,0
1,4,1,2017-12-19 10:06:55,2017-12-19 10:06:55,"This is an example of a page. Unlike posts, pa...",Page,NaN,publish,closed,closed,NaN,page,NaN,NaN,2017-12-29 02:51:46,2017-12-29 02:51:46,NaN,238,https://teknologi369969046.wordpress.com/?page...,2,page,NaN,0
2,5,1,2017-12-20 10:15:10,2017-12-20 10:15:10,<label> Your Name (required)\n [text* your-...,Contact form 1,NaN,publish,closed,closed,NaN,contact-form-1,NaN,NaN,2017-12-20 10:15:10,2017-12-20 10:15:10,NaN,0,http://dev.sagara.asia/teknologiid/?post_type=...,0,wpcf7_contact_form,NaN,0
3,7,1,2017-12-19 10:06:52,2017-12-19 10:06:52,This is the page where users will find your si...,Blog,NaN,publish,closed,closed,NaN,blog,NaN,NaN,2017-12-19 10:06:52,2017-12-19 10:06:52,NaN,0,https://teknologi369969046.wordpress.com/?page...,4,page,NaN,0
4,10,1,2017-09-22 09:28:22,2017-09-22 09:28:22,NaN,alex-blajan-227152,NaN,inherit,open,closed,NaN,alex-blajan-227152,NaN,NaN,2017-09-22 09:28:22,2017-09-22 09:28:22,NaN,0,https://manohara.incredibbble.com/wp-content/u...,0,attachment,image/jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4938,6456,1,2019-02-10 16:02:10,2019-02-10 09:02:10,NaN,wkwkwkwk,NaN,inherit,closed,closed,NaN,6452-revision-v1,NaN,NaN,2019-02-10 16:02:10,2019-02-10 09:02:10,NaN,6452,http://teknologiidberita.sagara.asia/publik/64...,0,revision,NaN,0
4939,6457,1,2019-02-10 16:02:16,2019-02-10 09:02:16,nnnn,wkwkwkwk,NaN,inherit,closed,closed,NaN,6452-revision-v1,NaN,NaN,2019-02-10 16:02:16,2019-02-10 09:02:16,NaN,6452,http://teknologiidberita.sagara.asia/publik/64...,0,revision,NaN,0
4940,6459,1,2019-04-02 10:35:47,0000-00-00 00:00:00,NaN,ini judul,NaN,draft,open,open,NaN,NaN,NaN,NaN,2019-04-02 10:35:47,2019-04-02 03:35:47,NaN,0,http://teknologiidberita.sagara.asia/?p=6459,0,post,NaN,0
4941,6461,1,2019-04-02 11:11:11,2019-04-02 04:11:11,Jangan lupa membaca berita teknologi indonesia...,Teknologi.id,NaN,publish,open,open,NaN,teknologi-id,NaN,NaN,2019-04-02 11:11:11,2019-04-02 04:11:11,NaN,0,http://teknologiidberita.sagara.asia/?p=6461,0,post,NaN,0


**Dari semua data artikel yang ada, hanya pilih artikel yang sudah di publish & bukan inherit dari artikel lain**

In [ ]:
datateknologi = datateknologi[datateknologi['post_status']=='publish'].reset_index(drop=True)
datateknologi

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,2,1,2017-12-20 10:13:01,2017-12-20 10:13:01,This is an example page. It's different from a...,Sample Page,NaN,publish,closed,open,NaN,sample-page,NaN,NaN,2017-12-20 10:13:01,2017-12-20 10:13:01,NaN,0,http://dev.sagara.asia/teknologiid/?page_id=2,0,page,NaN,0
1,4,1,2017-12-19 10:06:55,2017-12-19 10:06:55,"This is an example of a page. Unlike posts, pa...",Page,NaN,publish,closed,closed,NaN,page,NaN,NaN,2017-12-29 02:51:46,2017-12-29 02:51:46,NaN,238,https://teknologi369969046.wordpress.com/?page...,2,page,NaN,0
2,5,1,2017-12-20 10:15:10,2017-12-20 10:15:10,<label> Your Name (required)\n [text* your-...,Contact form 1,NaN,publish,closed,closed,NaN,contact-form-1,NaN,NaN,2017-12-20 10:15:10,2017-12-20 10:15:10,NaN,0,http://dev.sagara.asia/teknologiid/?post_type=...,0,wpcf7_contact_form,NaN,0
3,7,1,2017-12-19 10:06:52,2017-12-19 10:06:52,This is the page where users will find your si...,Blog,NaN,publish,closed,closed,NaN,blog,NaN,NaN,2017-12-19 10:06:52,2017-12-19 10:06:52,NaN,0,https://teknologi369969046.wordpress.com/?page...,4,page,NaN,0
4,41,1,2017-07-16 05:03:16,2017-07-16 05:03:16,"<figure class=""wp-caption""><img src=""http://de...","Gigih Septianto, Founder WeCare.id: Founder St...",NaN,publish,open,open,NaN,gigih-septianto-founder-wecare-id-founder-star...,NaN,NaN,2017-12-29 08:17:19,2017-12-29 08:17:19,NaN,0,https://teknologi369969046.wordpress.com/2017/...,0,post,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,6396,1,2018-10-17 13:37:37,2018-10-17 06:37:37,[newsletters_management],Manage Subscriptions,NaN,publish,closed,closed,NaN,manage-subscriptions,NaN,NaN,2018-10-17 13:37:37,2018-10-17 06:37:37,NaN,0,http://teknologiidberita.sagara.asia/manage-su...,0,page,NaN,0
806,6440,1,2019-02-10 15:18:20,2019-02-10 08:18:20,tes fcmtes fcmtes fcmtes fcmtes fcmtes fcmtes ...,tes fcm,NaN,publish,open,open,NaN,tes-fcm,NaN,NaN,2019-02-10 16:01:34,2019-02-10 09:01:34,NaN,0,http://teknologiidberita.sagara.asia/?p=6440,0,post,NaN,0
807,6452,1,2019-02-10 16:02:10,2019-02-10 09:02:10,nnnn,wkwkwkwk,NaN,publish,open,open,NaN,wkwkwkwk,NaN,NaN,2019-02-10 16:02:16,2019-02-10 09:02:16,NaN,0,http://teknologiidberita.sagara.asia/?p=6452,0,post,NaN,0
808,6454,1,2019-02-10 16:02:00,2019-02-10 09:02:00,coba coba,cobaa,NaN,publish,open,open,NaN,cobaa,NaN,NaN,2019-02-10 16:02:00,2019-02-10 09:02:00,NaN,0,http://teknologiidberita.sagara.asia/?p=6454,0,post,NaN,0


In [ ]:
datateknologi[datateknologi['post_parent']!=0]

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
1,4,1,2017-12-19 10:06:55,2017-12-19 10:06:55,"This is an example of a page. Unlike posts, pa...",Page,NaN,publish,closed,closed,NaN,page,NaN,NaN,2017-12-29 02:51:46,2017-12-29 02:51:46,NaN,238,https://teknologi369969046.wordpress.com/?page...,2,page,NaN,0
102,238,1,2017-09-23 09:50:14,2017-09-23 09:50:14,"<p class=""p1"">[tailor_section padding=""60px,,6...",Home,NaN,publish,closed,closed,NaN,home,NaN,NaN,2018-07-16 04:53:55,2018-07-16 04:53:55,NaN,765,https://manohara.incredibbble.com/?page_id=238,2,page,NaN,0
289,1287,1,2017-10-13 00:07:37,2017-10-13 00:07:37,<!-- tailor:tailor_section:59f05ca2b2574 -->[t...,Home 4,NaN,publish,closed,closed,NaN,home-4,NaN,NaN,2017-10-13 00:07:37,2017-10-13 00:07:37,NaN,238,https://manohara.incredibbble.com/?page_id=1287,0,page,NaN,0
290,1319,1,2017-10-13 00:44:23,2017-10-13 00:44:23,<!-- tailor:tailor_section:59f05d9b5c978 -->[t...,Home 5,NaN,publish,closed,closed,NaN,home-5,NaN,NaN,2017-10-13 00:44:23,2017-10-13 00:44:23,NaN,238,https://manohara.incredibbble.com/?page_id=1319,0,page,NaN,0
301,1529,1,2017-09-27 03:46:43,2017-09-27 03:46:43,<!-- tailor:tailor_section:59f059d967d8e -->[t...,Home 2,NaN,publish,closed,closed,NaN,home-2,NaN,NaN,2017-09-27 03:46:43,2017-09-27 03:46:43,NaN,238,https://manohara.incredibbble.com/?page_id=505,0,page,NaN,0
304,1544,1,2017-09-29 13:37:29,2017-09-29 13:37:29,<!-- tailor:tailor_section:59f05b032fc0i -->[t...,Home 3,NaN,publish,closed,closed,NaN,home-3,NaN,NaN,2017-09-29 13:37:29,2017-09-29 13:37:29,NaN,238,https://manohara.incredibbble.com/?page_id=962,0,page,NaN,0


In [ ]:
datateknologi = datateknologi[datateknologi['post_parent']==0].reset_index(drop=True)
datateknologi

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,2,1,2017-12-20 10:13:01,2017-12-20 10:13:01,This is an example page. It's different from a...,Sample Page,NaN,publish,closed,open,NaN,sample-page,NaN,NaN,2017-12-20 10:13:01,2017-12-20 10:13:01,NaN,0,http://dev.sagara.asia/teknologiid/?page_id=2,0,page,NaN,0
1,5,1,2017-12-20 10:15:10,2017-12-20 10:15:10,<label> Your Name (required)\n [text* your-...,Contact form 1,NaN,publish,closed,closed,NaN,contact-form-1,NaN,NaN,2017-12-20 10:15:10,2017-12-20 10:15:10,NaN,0,http://dev.sagara.asia/teknologiid/?post_type=...,0,wpcf7_contact_form,NaN,0
2,7,1,2017-12-19 10:06:52,2017-12-19 10:06:52,This is the page where users will find your si...,Blog,NaN,publish,closed,closed,NaN,blog,NaN,NaN,2017-12-19 10:06:52,2017-12-19 10:06:52,NaN,0,https://teknologi369969046.wordpress.com/?page...,4,page,NaN,0
3,41,1,2017-07-16 05:03:16,2017-07-16 05:03:16,"<figure class=""wp-caption""><img src=""http://de...","Gigih Septianto, Founder WeCare.id: Founder St...",NaN,publish,open,open,NaN,gigih-septianto-founder-wecare-id-founder-star...,NaN,NaN,2017-12-29 08:17:19,2017-12-29 08:17:19,NaN,0,https://teknologi369969046.wordpress.com/2017/...,0,post,NaN,0
4,42,1,2017-07-17 09:10:56,2017-07-17 09:10:56,"<figure><img src=""http://dev.sagara.asia/tekno...","Habibi Garden, Startup Asal Indonesia di Bidan...",NaN,publish,open,open,NaN,habibi-garden-startup-asal-indonesia-di-bidang...,NaN,NaN,2018-02-28 09:43:05,2018-02-28 09:43:05,NaN,0,http://dev.sagara.asia/teknologiid/2017/07/17/...,0,post,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,6396,1,2018-10-17 13:37:37,2018-10-17 06:37:37,[newsletters_management],Manage Subscriptions,NaN,publish,closed,closed,NaN,manage-subscriptions,NaN,NaN,2018-10-17 13:37:37,2018-10-17 06:37:37,NaN,0,http://teknologiidberita.sagara.asia/manage-su...,0,page,NaN,0
800,6440,1,2019-02-10 15:18:20,2019-02-10 08:18:20,tes fcmtes fcmtes fcmtes fcmtes fcmtes fcmtes ...,tes fcm,NaN,publish,open,open,NaN,tes-fcm,NaN,NaN,2019-02-10 16:01:34,2019-02-10 09:01:34,NaN,0,http://teknologiidberita.sagara.asia/?p=6440,0,post,NaN,0
801,6452,1,2019-02-10 16:02:10,2019-02-10 09:02:10,nnnn,wkwkwkwk,NaN,publish,open,open,NaN,wkwkwkwk,NaN,NaN,2019-02-10 16:02:16,2019-02-10 09:02:16,NaN,0,http://teknologiidberita.sagara.asia/?p=6452,0,post,NaN,0
802,6454,1,2019-02-10 16:02:00,2019-02-10 09:02:00,coba coba,cobaa,NaN,publish,open,open,NaN,cobaa,NaN,NaN,2019-02-10 16:02:00,2019-02-10 09:02:00,NaN,0,http://teknologiidberita.sagara.asia/?p=6454,0,post,NaN,0


**Cek ID apakah ada yang duplikat/tidak**

In [ ]:
datateknologi['ID'].value_counts()

4094    1
4554    1
473     1
4568    1
471     1
       ..
5222    1
3170    1
3169    1
3168    1
6145    1
Name: ID, Length: 804, dtype: int64

**Melihat isi artikel**

In [ ]:
datateknologi['post_title']

0                                            Sample Page
1                                         Contact form 1
2                                                   Blog
3      Gigih Septianto, Founder WeCare.id: Founder St...
4      Habibi Garden, Startup Asal Indonesia di Bidan...
                             ...                        
799                                 Manage Subscriptions
800                                              tes fcm
801                                             wkwkwkwk
802                                                cobaa
803                                         Teknologi.id
Name: post_title, Length: 804, dtype: object

In [ ]:
datateknologi['post_content']

0      This is an example page. It's different from a...
1      <label> Your Name (required)\n    [text* your-...
2      This is the page where users will find your si...
3      <figure class="wp-caption"><img src="http://de...
4      <figure><img src="http://dev.sagara.asia/tekno...
                             ...                        
799                             [newsletters_management]
800    tes fcmtes fcmtes fcmtes fcmtes fcmtes fcmtes ...
801                                                 nnnn
802                                            coba coba
803    Jangan lupa membaca berita teknologi indonesia...
Name: post_content, Length: 804, dtype: object

**Dari konten artikel yang ada di bawah, terlihat bahwa text masih kotor (tidak rapi)**

In [ ]:
datateknologi['post_content'][100]

'![](http://dev.sagara.asia/teknologiid/wp-\ncontent/uploads/2017/12/b327f-0vwpj13jfwndqezt9.png)Foto: Rollingstone.com\nBagi para penikmat film horor supranatural hit, Paranormal Activity, pasti\nsebelumnya sudah mendengar kabar bahwa film ini sedang dikembangkan menjadi\n_game_ berbasis _Virtual Reality_ (VR). Dalam beberapa tahun terakhir, projek\n_game_ ini sedang dikembangkan oleh studio VRWERX dan akan diberi judul\nParanormal Activity: The Lost Soul. Menurut rencanya, _game_ ini akan di rilis\ndi Playstation VR namun setelah sekian lama belum ada konfirmasi lebih lanjut\nmengenai kabar pastinya. Dan akhirnya, setelah di tunggu-tunggu sekian lama,\nbeberapa hari yang lalu situs IGN resmi melaporkan bahwa game ini akan dirilis\npada 15 Agustus 2017 mendatang di Playstation VR. Dalam g _ame_ ini nantinya\npemain akan ada berada di dalam rumah tua yang sudah lama ditinggalkan dan\nselanjutnya pemain harus memecahkan misteri-misteri supranatural yang ada\ndalam rumah tersebut. Sejauh

**Melihat kerapian text apabila menggunakan library html2text**

In [ ]:
print(html2text.html2text(datateknologi['post_content'][100]))

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/b327f-0vwpj13jfwndqezt9.png)Foto: Rollingstone.com
Bagi para penikmat film horor supranatural hit, Paranormal Activity, pasti
sebelumnya sudah mendengar kabar bahwa film ini sedang dikembangkan menjadi
_game_ berbasis _Virtual Reality_ (VR). Dalam beberapa tahun terakhir, projek
_game_ ini sedang dikembangkan oleh studio VRWERX dan akan diberi judul
Paranormal Activity: The Lost Soul. Menurut rencanya, _game_ ini akan di rilis
di Playstation VR namun setelah sekian lama belum ada konfirmasi lebih lanjut
mengenai kabar pastinya. Dan akhirnya, setelah di tunggu-tunggu sekian lama,
beberapa hari yang lalu situs IGN resmi melaporkan bahwa game ini akan dirilis
pada 15 Agustus 2017 mendatang di Playstation VR. Dalam g _ame_ ini nantinya
pemain akan ada berada di dalam rumah tua yang sudah lama ditinggalkan dan
selanjutnya pemain harus memecahkan misteri-misteri supranatural yang ada
dalam rumah tersebut. Sejauh ini sudah ada

In [ ]:
print(html2text.html2text(datateknologi['post_content'][97]))

[caption id="attachment_2352" align="alignnone"
width="770"]![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/09/Google-Maps-Gets-More-Social-With-Shared-
Lists-1024x634.jpg) Foto: GeoMarketing[/caption] Badan Informasi Geospasial
(BIG) berencana untuk merancang aplikasi peta digital. Aplikasi ini nantinya
dapat berfungsi untuk memantau kondisi jalanan, rambu lalu lintas, area
konstruksi, tata ruang, hingga pengembangan ekonomi mikro. Menurut Hasanudin
Zaenal Abidin, selaku Kepala BIG, aplikasi ini dapat digunakan untuk mengecek
situasi macet dan juga lokasi tujuan. Menurutnya, sudah ada perusahaan dalam
negeri yang mengaku berminat menjadi mitra untuk menjalankan proyek tersebut.
Saat ini, kedua belah pihak tengah menyusun nota kesepahaman (MoU). Namun,
Hasanuddin belum bisa menyebutkan nama perusahaan yang berminat menjadi
mitranya itu. Ia memastikan, perusahaan dalam negeri ini sudah berpengalaman.
Munculnya ide merancang aplikasi ini berawal dari banyaknya masyarakat


In [ ]:
print(html2text.html2text(datateknologi['post_content'][96]))

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/a1e73-1qkksxrygblwf__pxoxpgda.jpeg)Foto: mylph.com
Hanya dalam waktu kurang dari seperempat abad yang akan datang, chip memori
akan mengalahkan kemampuan pengolahan otak pada manusia. Hal tersebut di
utarakan oleh salah satu produsen chip dan semikonduktor terbesar di dunia,
yaitu Samsung Electronics. Dr Kim Ki-Nam selaku Presiden Bisnis Semikonduktor
di Divisi _Device Solutions_ (DS) Samsung Electronics menuturkan bahwa “Pada
20 tahun yang akan datang, _Solid State Drive_ akan mampu menampung semua
informasi yang ada pada otak manusia.” _Solid State Drive_ (SSD) dengan
pergerakannya yang begitu cepat telah menggantikan _Hard-Disk Drive_ (HDD)
bagi penyimpanan di PC ataupun Note Book pada berbagai pusat big data
terkemuka. Dr. Kim juga mencoba mendefinisikan revolusi industri keempat
sebagai penggabungan digital dari dunia fisik dan cyber. “Industri chip akan
membawa perubahan yang belum pernah ada sebelumnya. Baik itu 

In [ ]:
print(html2text.html2text(datateknologi['post_content'][95]))

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/4ff4f-1athpeh8lvhynjjetzqxlvq.jpeg)Foto:
trendhunter.com Aplikasi desain interior bernama Planner5D adalah sebuah
aplikasi yang memberikan solusi dalam mendesain sebuah ruangan dan telah
dilengkapi dengan kecerdasan buatan/ _artificial intelligence_ (AI) di
dalamnya. Keberadaan aplikasi Planner5D ini pun akan memberikan solusi yang
ringkas saat desain ruangan, semua aktivitas tersebut bisa dilakukan via layar
dan cukup dengan beberapa tombol klik. “Aplikasi ini berguna agar seseorang
tidak bergantung pada tenaga desainer interior” ucap _Co-founder_ Planner5D,
Alexey Sheremetyev. Aplikasi Planner5D ini memang masih belum bisa dijumpai,
karena masih dalam tahap pengembangan. Pihak _developer_ mengatakan kalau
algoritma pintar yang mereka kembangkan saat ini masih mempelajari sekitar 40
juta proyek dari para _real user_. AI tersebut pun akan dapat memberikan saran
mengenai penggunaan perabotan seperti lampu, kursi, serta p

In [ ]:
print(html2text.html2text(datateknologi['post_content'][94]))

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/a7154-1vw9gf9tttoemfdqwu9eazg.jpeg)Foto:
dailysocial.id Salah satu fitur terbaru pada iPhone X adalah Face ID, yaitu
sistem keamanan yang menggunakan wajah untuk membuka kunci pada iPhone X
tersebut. Dan ternyata Face ID tersebut berasal dari pengembang Kinect. Face
ID bekerja dengan menganalisa lebih dari 30.000 titik inframerah yang
diproyeksikan pada wajah pengguna guna membuat pemetaan wajah yang sangat
presisi. Chip A11 Bionic kemudian bertugas mengolah datanya secara aman dan
tanpa mengandalkan bantuan jaringan _cloud_ sama sekali. Sepintas, cara kerja
teknologi tersebut mirip dengan Kinect. Kinect adalah aksesoris untuk
_console_ Xbox yang bertugas membaca gerakan dan menerjemahkannya menjadi
_input_ kontrol. Kinect diumumkan pertama kali pada bulan Juni 2009, yang pada
awalnya menggunakan teknologi racikan sebuah perusahaan asal Israel bernama
PrimeSense. Teknologi itu melibatkan sebuah proyektor dan kamera infr

**Melihat apakah ada data yang NaN/null. Jika ada, isi data yang NaN dengan string kosong**

In [ ]:
datateknologi['ID'].isnull().values.any()

False

In [ ]:
datateknologi['post_content'].isnull().values.any()

True

In [ ]:
datateknologi['post_title'].isnull().values.any()

True

In [ ]:
datateknologi['post_content'] = datateknologi['post_content'].fillna('')
datateknologi['post_content'].isnull().values.any()

False

In [ ]:
datateknologi['post_title'] = datateknologi['post_title'].fillna('')
datateknologi['post_title'].isnull().values.any()

False

**Terapkan html2text preprocessing ke konten artikel**

In [ ]:
for i in range(len(datateknologi['post_content'])):
  datateknologi['post_content'][i] = html2text.html2text(datateknologi['post_content'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(datateknologi['post_content'][100])

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/50b0b-19is_hzcoxcydn2o1rkhutw.jpeg)Foto: kuow.org
Kanker pankreas telah menelan lebih dari 50.000 korban meninggal pada 2016 di
Amerika Serikat. Gejala penyakit ini baru bisa diketahui setelah sel kanker
tumbuh subur di dalam organ. Sulitnya mendeteksi kanker pankreas membuat tim
ilmuwan dari University of Washington merancang aplikasi _smartphone_ yang
bisa mendeteksi zat bilirubin lewat foto. Bilirubin sendiri adalah zat yang
menyebabkan mata menjadi berwarna kuning sebagai efek dari kanker pankreas.
Tim menjuluki aplikasi ini sebagai BiliScreen. Algoritma di dalamnya dapat
mengetahui kadar bilirubin sedari dini sehingga dapat segera dilakukan
penanganan sebelum terlambat. Shwetak Patel salah satu tim peneliti mengatakan
cara ini dapat memprediksi berapa banyak kadar bilirubin di dalam darah.
![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/07ffa-1wl9jm408qtbdyym5gyyxbg.png)Foto: tandaseru.id
Sebanya

**Merapikan konten artikel dengan:**


1.   **Mengubah semua karakter menjadi lowercase**
2.   **Mengganti \n dengan spasi**
3.   **Menghilangkan simbol dan angka**
4.   **Menghilangkan url link & gambar**
5.   **Menghilangkan stopwords (kata yang tidak mempunyai informasi seperti kata sambung) bahasa indonesia & inggris**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^a-z ]')
stopworda = set(stopwords.words('indonesian'))
stopwordb = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[\S]+\.(net|id|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?','',text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\S+.jpeg", "", text)
    text = re.sub(r"\S+.png", "", text)
    text = re.sub(r"\S+.jpg", "", text)
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    text = ' '.join(word for word in text.split() if word not in stopwordb)
    return text

datateknologi['post_contentCleaned'] = datateknologi['post_content'].apply(clean_text)

**Melihat hasil text**

In [ ]:
datateknologi['post_contentCleaned'][100]

'foto kanker pankreas menelan korban meninggal amerika serikat gejala penyakit sel kanker tumbuh subur organ sulitnya mendeteksi kanker pankreas tim ilmuwan university washington merancang aplikasi smartphone mendeteksi zat bilirubin foto bilirubin zat menyebabkan mata berwarna kuning efek kanker pankreas tim menjuluki aplikasi biliscreen algoritma dalamnya kadar bilirubin sedari penanganan terlambat shwetak patel salah tim peneliti memprediksi kadar bilirubin darah foto partisipan biliscreen kacamata frame khusus kalibrasi warna selfie syarat mata terbuka hasil penelitian pengukuran biliscreen mencapai level akurasi persen dibanding metode pengetesan darah tim berharap aplikasi orang mendeteksi kanker pankreas parah operasi meningkatkan usia harapan hidup penderita sumber dirangkum'

**Stemming (menghilangkan imbuhan) dari setiap kata dengan library Sastrawi. Proses ini memakan waktu lama (22 menit)**

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming_preprocessing(text):
    return stemmer.stem(text)

In [ ]:
%%time
datateknologi['post_contentStemmed'] = datateknologi['post_contentCleaned'].apply(stemming_preprocessing)

CPU times: user 21min 24s, sys: 3.18 s, total: 21min 27s
Wall time: 21min 28s


**Melihat hasil stemming**

In [ ]:
datateknologi['post_contentStemmed'][100]

'foto kanker pankreas tel korban tinggal amerika serikat gejala sakit sel kanker tumbuh subur organ sulit deteksi kanker pankreas tim ilmuwan university washington rancang aplikasi smartphone deteksi zat bilirubin foto bilirubin zat sebab mata warna kuning efek kanker pankreas tim juluk aplikasi biliscreen algoritma dalam kadar bilirubin dar tangan lambat shwetak patel salah tim teliti prediksi kadar bilirubin darah foto partisipan biliscreen kacamata frame khusus kalibrasi warna selfie syarat mata buka hasil teliti ukur biliscreen capai level akurasi persen banding metode ketes darah tim harap aplikasi orang deteksi kanker pankreas parah operasi tingkat usia harap hidup derita sumber rangkum'

**Set ID sebagai index untuk memudahkan pemrosesan**

In [ ]:
datateknologi.set_index('ID', inplace=True)

In [ ]:
indices = pd.Series(datateknologi.index)
indices[:50]

0       2
1       5
2       7
3      41
4      42
5      43
6      44
7      45
8      46
9      47
10     48
11     49
12     50
13     51
14     52
15     53
16     54
17     55
18     56
19     57
20     58
21     59
22     60
23     61
24     62
25     63
26     64
27     65
28     66
29     67
30     68
31     70
32     72
33     75
34     77
35     79
36     81
37     83
38     85
39     91
40     96
41    100
42    102
43    105
44    106
45    109
46    110
47    112
48    114
49    116
Name: ID, dtype: int64

##Pembuatan model Recommendation System

**Menghitung TF-IDF dari tiap artikel. TF-IDF menghitung bobot nilai setiap kata yang ada di artikel, semakin sering kata tersebut muncul di artikel semakin tinggi bobot nilainya.**

In [ ]:
stopwords_list = stopwords.words('indonesian')

vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

tfidf_matrix = vectorizer.fit_transform(datateknologi['post_title'] + " " + datateknologi['post_contentStemmed'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<804x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 73278 stored elements in Compressed Sparse Row format>

**Membuat matriks TF-IDF lalu menghitung cosine similarity. Cosine similarity menghitung kesamaan TF-IDF antara dua artikel. Semakin mirip isi konten artikel, semakin tinggi nilai cosine similaritynya (skala 0-1)**

In [ ]:
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.00420204, 0.25659344, ..., 0.        , 0.        ,
        0.        ],
       [0.00420204, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.25659344, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

**Membuat sistem rekomendasi dari nilai cosine similarity yang sudah dibuat.**

In [ ]:
def recommendations(id, cos_sim = cos_sim):
    recommended = []
    idx = indices[indices == id].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended.append(list(datateknologi.index)[i])
        
    return list(zip(recommended,score_series.iloc[1:11]))

**Contoh rekomendasi apabila pembaca membaca artikel dengan ID 100**

In [ ]:
recommendations(100)

[(3672, 0.4368507791065899),
 (712, 0.3966403113607972),
 (3666, 0.39359471552463293),
 (306, 0.3556067568851411),
 (5832, 0.3475693963181011),
 (5400, 0.33330582184940793),
 (120, 0.32466831944850216),
 (5675, 0.3173099461244564),
 (2560, 0.3064619372843206),
 (176, 0.28914886161008946)]

**Dari hasil rekomendasi diatas, tiga artikel yang paling direkomendasikan adalah artikel dengan ID 3672, 712, dan 3666. Untuk mengecek apakah rekomendasi sudah sesuai, cek isi konten dengan ID 100, 3672, 712, dan 3666 apakah isinya mirip/tidak.**

In [ ]:
datateknologi['post_title'][100]

'Game VR “Paranormal Activity: The Lost Soul” Akan di Rilis Oleh Playstation VR'

In [ ]:
print(datateknologi['post_content'][100])

![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2017/12/b327f-0vwpj13jfwndqezt9.png)Foto: Rollingstone.com
Bagi para penikmat film horor supranatural hit, Paranormal Activity, pasti
sebelumnya sudah mendengar kabar bahwa film ini sedang dikembangkan menjadi
_game_ berbasis _Virtual Reality_ (VR). Dalam beberapa tahun terakhir, projek
_game_ ini sedang dikembangkan oleh studio VRWERX dan akan diberi judul
Paranormal Activity: The Lost Soul. Menurut rencanya, _game_ ini akan di rilis
di Playstation VR namun setelah sekian lama belum ada konfirmasi lebih lanjut
mengenai kabar pastinya. Dan akhirnya, setelah di tunggu-tunggu sekian lama,
beberapa hari yang lalu situs IGN resmi melaporkan bahwa game ini akan dirilis
pada 15 Agustus 2017 mendatang di Playstation VR. Dalam g _ame_ ini nantinya
pemain akan ada berada di dalam rumah tua yang sudah lama ditinggalkan dan
selanjutnya pemain harus memecahkan misteri-misteri supranatural yang ada
dalam rumah tersebut. Sejauh ini sudah ada

In [ ]:
datateknologi['post_title'][3672]

'Inilah 5 Game Terbaik yang Akan Datang di Tahun Ini'

In [ ]:
print(datateknologi['post_content'][3672])

[caption id="attachment_3673" align="alignnone"
width="770"]![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2018/03/red-1024x576.jpg) Foto: egmnow.com[/caption]
[Venturebeat](https://venturebeat.com/) membuat daftar _game _yang kamu
butuhkan untuk ditunggu selama sisa tahun ini. Berikut 5 _game _terbaik yang
akan datang di sepanjang tahun 2018!

### **God of War**

  [caption id="attachment_3674" align="alignnone"
width="770"]![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2018/03/gow2-1024x576.jpg) Foto: wccftech.com[/caption]

Platform: PlayStation 4.

Tanggal rilis: 20 April 2018.

Terakhir kali Sony meluncurkan God of War: Ascension untuk PlayStation 3.
Namun versi tersebut tidak mendapatkan respon yang baik. Dan sekarang, serial
ini mendapatkan “requel” — salah satu sekuel reboot yang populer di film
sekarang — dan ini terlihat fantastis. Kratos telah pensiun dan membesarkan
anak-anak, dan kamu bisa menghabiskan sepanjang waktu membimbing anak tersebut
melalu

In [ ]:
datateknologi['post_title'][712]

'Game Ini Mengajak Pemain Menjadi SuperHero Sungguhan Menggunakan VR!'

In [ ]:
print(datateknologi['post_content'][712])

![](https://cdn-
images-1.medium.com/max/800/1*moCUiHn6rtTT2a4oFNc9Ow.jpeg)Venturebeat.com
Tidak seperti game pada umumnya, game ini mengajak anda menjadi salah satu
karakter SuperHero marve seperti Hulk, Mrs Marvel ataupun Rockie. Game ini
berjudl _Marvel Power United VR_ , game ini sedang dibuat oleh Vendor VR yaitu
Oculus dan Sanzaru Games. Pengumuman tersebut muncul saat panel game “Level
Up” juga yang menyertakan update tentang Spiderman, StarWars: Battlefront II
dan Insomniac Game. Mungkin tidak mengherankan, Game Ini akan eksklusif untuk
headset _Oculus Rift_ dan _Touch Controllers._ Salah satu nilai jualnya yang
besar adalah Pemain akan berpengalan bekerja sama dengan peran yang berbeda-
beda untuk misi yang sama. Game ini ditargetkan rilis pada tahun 2018.
![](https://cdn-
images-1.medium.com/max/800/1*VjGKTeUVnnKvEjEtFCQX1g.jpeg)digitalagencynetwork.com
Dengan Ini Marvel dan Oculus telah menemukan kemitraan yang memungkinkanya
menjadikan nama besar lain yang eksekutif untuk m

In [ ]:
datateknologi['post_title'][3666]

'Monster Hunter: World, Kandidat Kuat Game of The Year 2018'

In [ ]:
print(datateknologi['post_content'][3666])

[caption id="attachment_3667" align="alignnone"
width="770"]![](http://dev.sagara.asia/teknologiid/wp-
content/uploads/2018/03/mh-1024x640.jpg) Game Monster Hunter: World (foto:
telegaph.co.uk).[/caption]

Monster Hunter: World telah dirilis. Peluncuran _game action role-playing _ini
menuai respon yang positif dari para gamer dan juga media. Diberitakan Capcom
mendapatkan banyak keuntungan berkat peluncuran ini, dan juga Monster Hunter:
World berpeluang besar merebut gelar _Game of the Year_. Simak ulasan Monster
Hunter: World dibawah ini.

Polygon memberi skor 9, dan menjelaskan bahwa game ini merupakan game yang
kompleks dengan skenario yang tak bisa diprediksi. Namun, game ini dinilai
mampu memuaskan berbagai macam jenis gamer.

IGN dan Game Informer juga menyodorkan skor yang tinggi, yakni 9,5. IGN
menyebut bahwa game ini merupakan game Monster Hunter yang paling berani
dengan pertempuran yang adiktif dan monster-monster yang sangat
mengintimidasi. Sementara Game Informer menyebut 

**Dari rekomendasi diatas, terlihat bahwa sistem rekomendasi berhasil merekomendasikan artikel yang mirip. Semua artikel membahas game yang dirilis di Playstation**